In [8]:
# 0. Import libraries
import pandas as pd
import numpy as np

In [9]:
# 1. File path
file_path = r"C:\Users\gabri\Downloads\FLATPAY AUG25 STOCK MOVEMENT REPORT.xlsx"

In [10]:
# 2. Load the two shipment sheets
dk = pd.read_excel(file_path, sheet_name="Aug25 DK SHIPMENTS")
de = pd.read_excel(file_path, sheet_name="Aug25 DE SHIPMENTS")

print("DK Shipments Columns:", dk.columns.tolist())
print("DE Shipments Columns:", de.columns.tolist())


DK Shipments Columns: ['GoodsOwner', 'OrderNumber', 'ShippingAddress_Country', 'PackTime', 'ItemNumber', 'Description', 'QuantityPacked', 'ShippingAddress_CustomerName']
DE Shipments Columns: ['GoodsOwner', 'OrderNumber', 'ShippingAddress_Country', 'PickingWarehouseCountry', 'PackTime', 'ItemNumber', 'Description', 'QuantityPacked', 'ShippingAddress_CustomerName']


In [11]:
# 3. Function to standardize shipment columns
def standardize_shipments(df: pd.DataFrame, source_country_code: str) -> pd.DataFrame:
    # Rename columns to a common schema
    rename_map = {
        "OrderNumber": "OrderNumber",
        "ItemNumber": "ItemNumber",
        "Description": "Description",
        "QuantityPacked": "QuantityPacked",
        "PackTime": "PackTime",
        "ShippingAddress_Country": "DestinationCountry",
        "ShippingAddress_CustomerName": "CustomerName",
        "PickingWarehouseCountry": "WarehouseCountry",  # only DE has this
        "GoodsOwner": "GoodsOwner"
    }
    df = df.rename(columns=rename_map)

    # Ensure all standard columns exist
    standard_cols = [
        "OrderNumber","ItemNumber","Description","QuantityPacked","PackTime",
        "DestinationCountry","CustomerName","WarehouseCountry","GoodsOwner"
    ]
    for c in standard_cols:
        if c not in df.columns:
            df[c] = np.nan

    # Add source (where the data came from)
    df["SourceCountry"] = source_country_code

    # If WarehouseCountry missing, assume it equals source country
    df["WarehouseCountry"] = df["WarehouseCountry"].fillna(source_country_code)

    # Clean datatypes
    df["PackTime"] = pd.to_datetime(df["PackTime"], errors="coerce")
    df["QuantityPacked"] = pd.to_numeric(df["QuantityPacked"], errors="coerce")

    # Meta tags for flows
    df["DataType"] = "Flow"
    df["FlowDirection"] = "Out"
    df["FlowType"] = "CustomerShipment"

    # Keep columns in order
    ordered = [
        "SourceCountry","WarehouseCountry","OrderNumber","PackTime","ItemNumber",
        "Description","CustomerName","DestinationCountry","QuantityPacked",
        "GoodsOwner","DataType","FlowDirection","FlowType"
    ]
    return df[ordered]


In [12]:
# 4. Standardize both DK and DE
dk_std = standardize_shipments(dk.copy(), "DK")
de_std = standardize_shipments(de.copy(), "DE")


In [13]:
# 5. Combine into one dataset
shipments_all = pd.concat([dk_std, de_std], ignore_index=True)

# Quick preview
shipments_all.head(10)


,SourceCountry,WarehouseCountry,OrderNumber,PackTime,ItemNumber,Description,CustomerName,DestinationCountry,QuantityPacked,GoodsOwner,DataType,FlowDirection,FlowType
0,DK,DK,FP-2399,2025-08-22 10:07:22.710103300,0510-00-4-S-W#FP,FR-Classic-Tshirt-(Herre)-S-W,Zakaria Benfadel,FR,1.0,Flatpay,Flow,Out,CustomerShipment
1,DK,DK,FP-2399,2025-08-22 10:07:34.086965700,9630-99-4-S-B#FP,FR-Sort-Jakke-Unisex-(Herre)-S-B,Zakaria Benfadel,FR,1.0,Flatpay,Flow,Out,CustomerShipment
2,DK,DK,FP-2399,2025-08-22 10:07:28.897835400,021033-99-4-S-B#FP,FR-Basic-Halfzip-Unisex-S-B,Zakaria Benfadel,FR,1.0,Flatpay,Flow,Out,CustomerShipment
3,DK,DK,FP-2399,2025-08-22 10:07:25.961197700,029033-00-4-S-W#FP,FR-Basic-Langærmet-(Herre)-S-W,Zakaria Benfadel,FR,1.0,Flatpay,Flow,Out,CustomerShipment
4,DK,DK,FP-2399,2025-08-22 10:07:32.134619000,5430-99-4-S-B#FP,FR-Sort-Hoodie-Unisex-S-B,Zakaria Benfadel,FR,1.0,Flatpay,Flow,Out,CustomerShipment
5,DK,DK,FP-2399,2025-08-22 10:07:36.269112200,020931-99-4-B-S#FP,FR-WintherJacket-Men-S,Zakaria Benfadel,FR,1.0,Flatpay,Flow,Out,CustomerShipment
6,DK,DK,FP-2005,2025-08-15 10:40:52.585307500,VL305.741#FP,Receipt paper for terminals - 60 RLL,Zafer Coban,DE,1.0,Flatpay,Flow,Out,CustomerShipment
7,DK,DK,FP-2005,2025-08-15 10:40:52.578051500,VL308.096#FP,Receipt paper for POS - 30 RLL,Zafer Coban,DE,1.0,Flatpay,Flow,Out,CustomerShipment
8,DK,DK,FP-2005,2025-08-15 10:40:52.578051500,VL308.096#FP,Receipt paper for POS - 30 RLL,Zafer Coban,DE,1.0,Flatpay,Flow,Out,CustomerShipment
9,DK,DK,FP-2005,2025-08-19 09:26:13.907297100,VL308.096#FP,Receipt paper for POS - 30 RLL,Zafer Coban,DE,1.0,Flatpay,Flow,Out,CustomerShipment


In [14]:
# 6. Save results
shipments_all.to_csv("Shipments_Consolidated.csv", index=False)
shipments_all.to_excel("Shipments_Consolidated.xlsx", index=False)

print("✅ Saved Shipments_Consolidated.csv and Shipments_Consolidated.xlsx")


✅ Saved Shipments_Consolidated.csv and Shipments_Consolidated.xlsx
